In [1]:
import os
os.chdir('..')

# Import Files

In [2]:
from src.agent_interface import *
from src.agent import *
from src.env import *
from src.price_data import *
from src.prior_measure import *
from src.q import *
from src.robust import *
from src.train import *
from src.util import *

In [3]:
checkpoint_file_name = "checkpoint_ep5.pt"
runtime_folder = "20260222_124228"

In [4]:
c = Config()
model_name = "TEST_MODEL"
config_path = c.build_config_path(model_name)
stock_params, env_params, training_controller_params, duality_params, q_params, dqn_params, other_params = c.load_config(config_path)

In [5]:
yf = YahooFinance(**stock_params)
df = yf.pipeline()
asset_log_returns = df['log_return'].dropna().to_numpy()
env = PortfolioEnv(asset_log_returns=asset_log_returns, **env_params)

[*********************100%***********************]  1 of 1 completed


In [ ]:
device = torch.device('cuda')
model_name, n_epochs = other_params.values()

training_controller = TrainingController(**training_controller_params)
prior_measure = PriorStudentDistribution(device=device)
duality_operator = DualityHQOperator(**duality_params)
q = QFunc(**q_params)
writer = PORDQNProgressWriter(model_name, timestamp_folder_name=runtime_folder, overwrite_existing_checkpoint_file=True)
agent = PORDQN(training_controller=training_controller, prior_measure=prior_measure, duality_operator=duality_operator,
               qfunc = q, writer=writer, device=device, **dqn_params)

In [7]:
load_model = LoadModel()
agent, current_epoch = load_model.restore_agent_state(agent, model_name, checkpoint_file_name, runtime_folder, target_device=device)

In [8]:
cum_rewards = train_agent(env, agent, current_epoch=6, n_epochs=7, writer=writer)

Episode 6: 100%|██████████| 7492/7492 [06:43<00:00, 18.56it/s]


Episode 6 mean of summed rewards: -2.598


Episode 7: 100%|██████████| 7492/7492 [04:37<00:00, 26.99it/s]


Episode 7 mean of summed rewards: -2.598
